In [1]:
using Random
using Random: bitrand
using Revise, GADNF

In [2]:
Random.seed!(81)

TaskLocalRNG()

## Individual

In [3]:
feature_names = ["A", "B", "C"]

3-element Vector{String}:
 "A"
 "B"
 "C"

In [4]:
ind = GADNF.Individual(3, 2)

2-element Vector{Bool}:
 1
 1

3×2 Matrix{Int8}:
 -1  -1
  0   1
 -1  -1

In [5]:
GADNF.mark_selected_regulators!(ind)

3-element BitVector:
 1
 1
 1

In [6]:
GADNF.get_selected_regulators(ind.regulator_selected, feature_names)

3-element Vector{String}:
 "A"
 "B"
 "C"

In [7]:
GADNF.to_expression(ind, feature_names)

"(~A & ~C) | (~A & B & ~C)"

In [8]:
GADNF.to_simplified_expression(ind, feature_names)

(!A & !C) | ((!A & B) & !C)

### mutate

In [9]:
GADNF.mutate!(ind)
ind

2-element Vector{Bool}:
 1
 1

3×2 Matrix{Int8}:
 -1  -1
  0   1
 -1  -1

In [10]:
GADNF.mutate!(ind)
ind

2-element Vector{Bool}:
 1
 1

3×2 Matrix{Int8}:
 -1  -1
  0   1
 -1  -1

### cx

In [11]:
ind1 = GADNF.Individual(3, 2)
ind2 = GADNF.Individual(3, 2)

display.((ind1.CNs, ind2.CNs))

3×2 Matrix{Int8}:
 1  -1
 1   1
 0   1

3×2 Matrix{Int8}:
  1  1
 -1  0
  0  1

(nothing, nothing)

In [12]:
GADNF.crossover!(ind1, ind2; edge_cx_rate=0.4)
display.((ind1.CNs, ind2.CNs))

3×2 Matrix{Int8}:
 1  1
 1  0
 0  1

3×2 Matrix{Int8}:
  1  -1
 -1   1
  0   1

(nothing, nothing)

### fitness

#### fitting error

In [13]:
ind1 = GADNF.Individual(3, 2)
GADNF.to_expression(ind1, feature_names)

"~A & ~B & ~C"

In [14]:
X = bitrand(3, 4)
y = bitrand(4)
display(X) 
display(y)

3×4 BitMatrix:
 0  0  1  1
 1  0  1  1
 0  0  0  0

4-element BitVector:
 1
 1
 1
 1

In [15]:
GADNF.count_fitting_error!(ind1, X, y)

3

#### complexity

In [16]:
ind1 = GADNF.Individual(3, 2)
display(ind1)

2-element Vector{Bool}:
 0
 0

3×2 Matrix{Int8}:
 -1  -1
  0   0
 -1   0

In [17]:
GADNF.mark_selected_regulators!(ind1)

3-element BitVector:
 0
 0
 0

In [18]:
GADNF.count_complexity(ind1)

0.0

## GA

### Random data

In [19]:
target = "target"
feature_names = ["A", "B", "C"]
num_inputs = length(feature_names)
num_samples= 6

6

In [20]:
X = bitrand(num_inputs, num_samples)
y = bitrand(num_samples)
display(X) 
display(y')

3×6 BitMatrix:
 1  0  0  0  0  1
 0  0  0  0  1  1
 0  1  0  1  0  0

1×6 adjoint(::BitVector) with eltype Bool:
 0  0  1  0  1  0

In [21]:
cfg = GAConfig(; num_generations=50, population_size=100, num_conjunctions=3,
    allowed_stagnation_generations=20, mut_rate=0.2, mut_rate_min=0.3,
    mut_rate_max=0.7)

GAConfig(50, 100, 1, 3, 2, 0.05, 0.05, 0.1, 0.1, 0.3, 0.7, 0.2, 20)

In [22]:
final_pop = run_GA(X, y; cfg, target)

Running GA for target ...
1     (0.1667, 1.2222)
2     (0.1667, 1.2222)
3     (0.0000, 2.2222)
4     (0.0000, 1.2222)
5     (0.0000, 1.2222)
6     (0.0000, 1.2222)
7     (0.0000, 1.2222)
8     (0.0000, 1.2222)
9     (0.0000, 1.2222)
10    (0.0000, 1.2222)
11    (0.0000, 1.2222)
12    (0.0000, 1.2222)
13    (0.0000, 1.2222)
14    (0.0000, 1.2222)
15    (0.0000, 1.2222)
16    (0.0000, 1.2222)
17    (0.0000, 1.2222)
18    (0.0000, 1.2222)
19    (0.0000, 1.2222)
20    (0.0000, 1.2222)
21    (0.0000, 1.2222)
22    (0.0000, 1.2222)
23    (0.0000, 1.2222)
24    (0.0000, 1.2222)
25    (0.0000, 1.2222)
26    (0.0000, 1.2222)
Early termination: 20 generations without improvement.


100-element Vector{Individual}:
 Individual(Bool[0, 0, 1], Int8[-1 -1 -1; 1 1 0; 0 1 -1], 0.0, 1.2222222222222223, Bool[0, 0, 1, 0, 1, 0], Bool[1, 0, 1])
 Individual(Bool[0, 0, 1], Int8[0 1 -1; -1 -1 0; 1 1 -1], 0.0, 1.2222222222222223, Bool[0, 0, 1, 0, 1, 0], Bool[1, 0, 1])
 Individual(Bool[0, 0, 1], Int8[0 -1 -1; -1 -1 0; 1 0 -1], 0.0, 1.2222222222222223, Bool[0, 0, 1, 0, 1, 0], Bool[1, 0, 1])
 Individual(Bool[1, 0, 1], Int8[0 -1 -1; -1 -1 0; 1 0 -1], 0.3333333333333333, 2.1111111111111107, Bool[0, 1, 1, 1, 1, 0], Bool[1, 1, 1])
 Individual(Bool[0, 0, 1], Int8[0 1 -1; -1 -1 0; 1 0 -1], 0.0, 1.2222222222222223, Bool[0, 0, 1, 0, 1, 0], Bool[1, 0, 1])
 Individual(Bool[0, 0, 1], Int8[1 -1 -1; 1 -1 0; 1 0 -1], 0.0, 1.2222222222222223, Bool[0, 0, 1, 0, 1, 0], Bool[1, 0, 1])
 Individual(Bool[0, 0, 1], Int8[0 0 -1; -1 0 0; 1 0 -1], 0.0, 1.2222222222222223, Bool[0, 0, 1, 0, 1, 0], Bool[1, 0, 1])
 Individual(Bool[0, 0, 1], Int8[-1 -1 -1; 1 0 0; 1 1 -1], 0.0, 1.2222222222222223, Bool[0, 0, 1, 0

In [23]:
best = minimum(final_pop)

3-element Vector{Bool}:
 0
 0
 1

3×3 Matrix{Int8}:
 -1  -1  -1
  1   1   0
  0   1  -1

In [24]:
to_expression(best, feature_names)

"~A & ~C"

### Synthetic dataset

In [41]:
# (A & B) | C
# f(x) = (x[1] & x[2]) | (x[2] & ~x[5])
f(x) = (x[2] & x[3]) | x[4]
num_inputs = 7
num_samples = 45
X, y = generate_bool_data(f, num_inputs, num_samples)
display.((X, y'));

7×45 BitMatrix:
 0  1  1  0  0  1  1  1  0  0  0  1  1  …  1  1  0  1  0  1  0  0  1  0  1  0
 1  1  0  1  1  1  0  0  1  1  0  1  0     0  1  1  1  0  1  0  1  1  0  0  0
 1  1  0  1  1  0  0  0  0  1  0  0  1     0  0  1  1  0  1  0  0  0  0  0  1
 1  1  1  1  0  1  1  0  0  0  0  1  1     0  0  0  0  0  0  0  1  1  0  0  1
 1  0  0  0  1  1  1  0  0  0  0  0  1     1  1  1  0  1  1  0  0  1  1  1  0
 0  0  0  0  0  1  1  0  1  1  1  0  0  …  1  0  1  0  1  1  1  1  1  1  1  1
 1  0  0  1  0  0  0  1  0  0  1  0  0     1  1  0  1  1  0  0  1  1  0  0  1

1×45 adjoint(::BitVector) with eltype Bool:
 1  1  1  1  1  1  1  0  0  1  0  1  1  …  0  0  1  1  0  1  0  1  1  0  0  1

In [42]:
cfg = GAConfig(; num_generations=100, population_size=100, num_conjunctions=3,num_elites=3,
    allowed_stagnation_generations=20,mut_rate=0.2, mut_rate_min=0.3,
    mut_rate_max=0.7)

GAConfig(100, 100, 3, 3, 2, 0.05, 0.05, 0.1, 0.1, 0.3, 0.7, 0.2, 20)

In [43]:
final_pop = run_GA(X, y; cfg);

Running GA for y ...
1     (0.4889, 0.9048)
2     (0.4444, 1.2381)
3     (0.4444, 1.2381)
4     (0.3333, 1.4286)
5     (0.3333, 1.4286)
6     (0.3333, 1.0476)
7     (0.3333, 1.0476)
8     (0.3111, 1.2381)
9     (0.2444, 1.4286)
10    (0.1778, 2.3333)
11    (0.1778, 2.2857)
12    (0.1778, 2.2857)
13    (0.1111, 1.2381)
14    (0.1111, 1.2381)
15    (0.1111, 1.2381)
16    (0.1111, 1.2381)
17    (0.1111, 1.2381)
18    (0.0889, 1.0476)
19    (0.0667, 1.4286)
20    (0.0667, 1.4286)
21    (0.0667, 1.4286)
22    (0.0000, 1.2381)
23    (0.0000, 1.2381)
24    (0.0000, 1.2381)
25    (0.0000, 1.2381)
26    (0.0000, 1.2381)
27    (0.0000, 1.2381)
28    (0.0000, 1.2381)
29    (0.0000, 1.2381)
30    (0.0000, 1.2381)
31    (0.0000, 1.2381)
32    (0.0000, 1.2381)
33    (0.0000, 1.2381)
34    (0.0000, 1.2381)
35    (0.0000, 1.2381)
36    (0.0000, 1.2381)
37    (0.0000, 1.2381)
38    (0.0000, 1.2381)
39    (0.0000, 1.2381)
40    (0.0000, 1.2381)
41    (0.0000, 1.2381)
42    (0.0000, 1.2381)
Early termina

In [44]:
best = minimum(final_pop)
to_expression(best)

"(x2 & x3) | (x4)"

In [45]:
to_simplified_expression(best)

(x2 & x3) | x4

In [47]:
get_fitness(best)

(0.0, 1.238095238095238)